# To-Do List Manager Agent

    Let's Build the To-Do List Manager Agent using logic and custom tools (file manipulation), but we'll clearly segment the code and explanation for each part, starting from scratch.

**1. Brain (LLM) 🧠**<br>
The Brain is the Large Language Model (LLM) that handles the reasoning. It takes the user's request and decides the best course of action: which tool to use, the exact arguments for that tool, or if the question can be answered directly.

| Part          |Role in To-Do Agent                                                                                    |
|---------------|-------------------------------------------------------------------------------------------------------|
| Brain (LLM)   |Decides if the user wants to Add_Item, View_List, or Clear_List based on their natural language input. |

In [4]:
# 💻 Code: Brain Setup
import os
from langchain_openai import ChatOpenAI

# Set your API key environment variable (e.g., in your terminal or IDE)
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"

# Initialize the LLM (The Brain)
# Setting temperature to 0 makes the model deterministic and reliable for tool use
llm_brain = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

print("✅ Brain (LLM) initialized using gpt-3.5-turbo.")

✅ Brain (LLM) initialized using gpt-3.5-turbo.


**2. Tools 🛠️**<br>
Tools are the external functions the agent can call. Since LLMs can't directly read/write files or manage persistent state, we create three custom Python functions and wrap them so the LLM can use them.

|Part	|Tool Name	|Function|
|-------|-----------|--------|
|Tools	|Add_Item	|Appends a task to the todo_list.txt file.|
|Tools	|View_List	|Reads and returns the entire content of the todo_list.txt file.|
|Tools	|Clear_List	|Empties the todo_list.txt file.|

In [5]:
# 💻 Code: Tools Setup
from langchain.agents import Tool

TODO_FILE = "todo_list.txt"

def add_to_list(item: str) -> str:
    """Adds a new single task/item to the to-do list file."""
    try:
        with open(TODO_FILE, "a") as f:
            f.write(item + "\n")
        return f"Successfully added '{item}' to the to-do list."
    except Exception as e:
        return f"Error adding item: {e}"

def view_list(query: str) -> str:
    """Reads and returns the current items in the todo list file."""
    try:
        if not os.path.exists(TODO_FILE) or os.path.getsize(TODO_FILE) == 0:
            return "The to-do list is currently empty."
        
        with open(TODO_FILE, "r") as f:
            content = f.read()
        return "Current To-Do List:\n" + content
    except Exception as e:
        return f"Error viewing list: {e}"

def clear_list(query: str) -> str:
    """Empties the entire contents of the todo list file."""
    try:
        with open(TODO_FILE, "w") as f:
            f.write("")
        return "To-do list cleared successfully."
    except Exception as e:
        return f"Error clearing list: {e}"

# List of Tools for the Agent
agent_tools = [
    Tool(
        name="Add_Item",
        func=add_to_list,
        description="Useful for adding a new task to the to-do list. Input must be the exact task string."
    ),
    Tool(
        name="View_List",
        func=view_list,
        description="Useful for showing all current tasks in the list. Input is a placeholder and is ignored."
    ),
    Tool(
        name="Clear_List",
        func=clear_list,
        description="Useful for deleting all tasks from the list. Input is a placeholder and is ignored."
    ),
]

print(f"✅ {len(agent_tools)} Tools defined and wrapped for file: {TODO_FILE}")

✅ 3 Tools defined and wrapped for file: todo_list.txt


**3. Instructions (Prompt) 📜**<br>
The Instructions define the agent's identity, rules, and the expected thought process (the ReAct format). We use a System Message to inject the persona and constraints into the LLM's initial context.

|Part	|Purpose|
|-------|-------|
|Instructions	|Sets the persona as a "diligent and helpful To-Do List Manager". Explicitly tells the agent to only use the provided tools for managing the list.|

In [6]:
# 💻 Code: Instructions Setup
from langchain import hub

# Define the System Message (Instructions)
SYSTEM_MESSAGE = (
    "You are a diligent and helpful To-Do List Manager Agent. "
    "Your primary goal is to use the provided tools to manage the user's to-do list stored in the file named 'todo_list.txt'. "
    "Use the tools when the user's intent is clearly to add, view, or clear the list. "
    "Be polite and confirm every action with the user."
)

# Pull the standard ReAct template from the LangChain hub
prompt_template = hub.pull("hwchase17/react")

# Customize the template by injecting the System Message
custom_prompt = prompt_template.partial(system_prompt=SYSTEM_MESSAGE)

print("✅ Instructions (Prompt Template) customized with Agent Persona.")

✅ Instructions (Prompt Template) customized with Agent Persona.


**4. Executor (Controller) 🕹️**<br>
The Executor is the engine that runs the entire agent loop. It takes the Brain, Tools, and Instructions, and manages the cyclical flow: LLM's Thought → Action → Tool Observation → back to the LLM.

|Part|	Role in To-Do Agent|
|----|---------------------|
|Executor (Controller)|	Combines the LLM and Tools into an executable ReAct chain, managing the file I/O operations and response generation.|

In [7]:
# 💻 Code: Executor Setup

from langchain.agents import create_react_agent, AgentExecutor

# Create the Agent Chain (The core logic/Agent Type)
todo_agent = create_react_agent(
    llm=llm_brain,
    tools=agent_tools,
    prompt=custom_prompt # Use our customized prompt
)

# Create the Executor (The Controller that runs the loop)
agent_executor = AgentExecutor(
    agent=todo_agent,
    tools=agent_tools,
    verbose=True, # Display the full Thought/Action/Observation loop
    handle_parsing_errors=True
)

print("✅ Executor (Controller) created and linked to the Brain, Tools, and Instructions.")

✅ Executor (Controller) created and linked to the Brain, Tools, and Instructions.


**5. Memory 🧠 (Optional)**<br>
Memory allows the agent to maintain context across multiple turns. Since the To-Do List Agent uses a file (todo_list.txt) to maintain state (what items are on the list), that file acts as its long-term task memory.

For conversational memory (remembering what was just said), we use a memory buffer. This is essential for handling follow-up questions or references.

|Part	|Purpose in To-Do Agent|
|-------|----------------------|
|Task Memory	|The todo_list.txt file provides state persistence (which tasks are active).|
Conversational Memory	|A ConversationBufferWindowMemory allows the agent to reference previous turns (e.g., "Add that to the list" after naming an item).|

<br><br>
To fully integrate memory into a ReAct agent, we need to adjust the prompt to accept the history. For simplicity here, we define the memory object and then show how it would be used if the agent's prompt were specifically designed for chat history.



In [ ]:
# 💻 Code: Memory Setup (Conversational)
from langchain.memory import ConversationBufferWindowMemory

# Initialize Conversational Memory
# We'll keep a memory of the last 5 exchanges (user input + agent output)
agent_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", 
    k=5, 
    return_messages=True 
)

print("✅ Conversational Memory initialized (Buffer Window, k=5).")

# --- Agent Execution Demonstration ---
# Note: For this specific ReAct Agent setup, direct Memory integration into 
# the AgentExecutor often requires a different prompt or method.
# For demo purposes, we will continue to use the simplified Executor.

print("\n--- To-Do List Agent Demo (Testing all parts) ---")

# Test 1: Add item
print("\n--- 1. User Input: Add an item ---")
response_add = agent_executor.invoke({"input": "I need to add 'Call the plumber' to my list."})
print(f"\n**Agent Final Response:** {response_add['output']}")

# Test 2: View list
print("\n--- 2. User Input: View the list ---")
response_view = agent_executor.invoke({"input": "What do I have on my to-do list?"})
print(f"\n**Agent Final Response:** {response_view['output']}")

# Test 3: Clear list
print("\n--- 3. User Input: Clear the list ---")
response_clear = agent_executor.invoke({"input": "Please wipe the list clean."})
print(f"\n**Agent Final Response:** {response_clear['output']}")